# Nonlinear Least-squares for Drone Localization in GPS-denied Environments

$ \Large \frac{\partial{f(x)}^T}{\partial{x}} \frac{\partial{f(x)}}{\partial{x}}
\Delta X = \frac{\partial{f(x)}^T}{\partial{x}} (b - f(x))$

$ \Large \Delta X = (\frac{\partial{f(x)}^T}{\partial{x}} \Sigma^{-1} \frac{\partial{f(x)}}{\partial{x}})^{-1} \frac{\partial{f(x)}^T}{\partial{x}} (b - f(x))$

$ \Large X_{t+1} = X_t - \Delta X_t $

In [15]:
import numpy as np
import matplotlib.pyplot as plt

In [16]:
def createPoints(angle=1, radius=1, num=1):
    deg2rad = np.pi / 180
    theta = deg2rad * np.random.randint(-angle, angle, size=num)
    phi = deg2rad * np.random.randint(-angle / 3 + 90, angle / 3 + 90, size=num)
    x = radius * np.cos(theta) * np.sin(phi)
    y = radius * np.sin(theta) * np.sin(phi)
    z = radius * np.cos(phi)
    points = np.vstack((x, y, z))
    return points


def plotSatellites(satellites, stations):
    %matplotlib qt

    _, num_satellites = np.shape(satellites)
    _, num_stations = np.shape(stations)

    plt.rcParams["figure.figsize"] = [6, 6]
    plt.rcParams["figure.autolayout"] = True
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")

    theta = np.linspace(0, 2.0 * np.pi, 100)
    phi = np.linspace(0, np.pi, 100)

    # Convert to Cartesian coordinates
    rad_earth = 6378
    x = rad_earth * np.outer(np.cos(theta), np.sin(phi))
    y = rad_earth * np.outer(np.sin(theta), np.sin(phi))
    z = rad_earth * np.outer(np.ones(np.size(theta)), np.cos(phi))
    ax.plot_surface(x, y, z, cmap=plt.cm.YlGnBu_r)

    ax.plot(satellites[0, :], satellites[1, :], satellites[2, :], "k.")
    ax.plot(stations[0, :], stations[1, :], stations[2, :], "k.")

    for i in range(num_satellites):
        for j in range(num_stations):
            ax.plot(
                [stations[0, j], satellites[0, i]],
                [stations[1, j], satellites[1, i]],
                [stations[2, j], satellites[2, i]],
                "--",
                c=[i / num_satellites, 1 - i / num_satellites, i / num_satellites],
            )

    ax.set_xlabel("x axis")
    ax.set_ylabel("y axis")
    ax.set_zlabel("z axis")
    ax.set_aspect("equal")
    plt.show()

    return ax

In [17]:
# dimensions
rad_earth = 6378
rad_orbit = rad_earth + 20350

# gound stations
num_stations = 5
stations = createPoints(angle=25, radius=rad_earth, num=num_stations)

# satellites
num_satellites = 2
satellites = createPoints(angle=50, radius=rad_orbit, num=num_satellites)

# plot ground truth
ax = plotSatellites(satellites, stations)

In [18]:
def jacobian(satellites, stations):
    def partial(satellite, station):
        dfdx = -(station - satellite) / np.linalg.norm(station - satellite)

        return dfdx

    _, num_sat = np.shape(satellites)
    _, num_stat = np.shape(stations)
    J = np.zeros((num_sat * num_stat, 3 * num_sat))
    for i in range(num_stat):
        for j in range(num_sat):
            J[i + num_stat * j, 3 * j : (3 * j + 3)] = partial(
                satellites[:, j], stations[:, i]
            )

    return J


J = jacobian(satellites, stations)

meanStation = np.mean(stations, axis=1)
X = np.tile(meanStation, (1, 2))
X = np.reshape(X, (3 * num_satellites, 1))
# print(X)
# print(satellites)
# print(stations)


def measurements(satellites, stations):
    y = np.zeros((np.shape(satellites)[1] * np.shape(stations)[1], 1))
    for i in range(np.shape(satellites)[1]):
        satellite = np.reshape(satellites[:, i], (3, 1))
        l2 = np.linalg.norm(satellite - stations, axis=0)
        y[
            (np.shape(stations)[1] * i) : (
                np.shape(stations)[1] * i + np.shape(stations)[1]
            )
        ] = np.reshape(l2, (np.shape(stations)[1], 1))
    return y


y = measurements(satellites, stations)
print(y)


def gaussNewton():
    def deltaX(J, b, fx):
        pinv = np.linalg.inv(J.T @ J)
        dX = pinv @ J.T @ (b - fx)
        return dX

    return

[[21768.97796834]
 [21058.65023126]
 [20514.61657745]
 [22473.85181445]
 [20946.43810009]
 [22581.08908434]
 [22079.27654622]
 [20474.81280709]
 [23927.92433447]
 [21562.8859727 ]]


# Drone Environment

In [19]:
def ground(x):
    y = 0.5 * np.cos(x) + 4 * np.cos(0.2 * x) + 0.51 * x - 0.005 * x**2
    # y = 0.15*np.cos(x) + 1
    # y = 0.15*x
    return y


def height2pressure(h, var=0):
    h0 = 0
    p0 = 1013.25
    g = 9.80665
    M = 0.0289644
    R = 8.31432
    T = 273

    noise = np.random.normal(0, scale=var)
    p = p0 * np.exp(-g * M * (h - h0) / R / T)

    return p + noise


def pressure2height(p, var=0):
    h0 = 0
    p0 = 1013.25
    g = 9.80665
    M = 0.0289644
    R = 8.31432
    T = 273

    noise = np.random.normal(0, scale=var)
    h = h0 - np.log(p / p0) * R * T / g / M

    return h + noise


def fx(state, x_old):
    x, y = state

    x_old = np.array([[x_old[0]], [x_old[1]]])
    A = np.eye(2)
    B = np.eye(2)
    est_u = np.linalg.inv(B.T @ B) @ B.T @ (np.array([[x], [y]]) - A @ x_old)

    f1 = height2pressure(y)
    f2 = y - ground(x)
    f3 = est_u[0, 0]
    f4 = est_u[1, 0]

    f = np.array([[f1], [f2], [f3], [f4]])

    return f


def partialf(state, x_old):
    x, y = state
    dx = 0.000001
    dy = 0.000001

    dfdx1 = (fx((x + dx, y), x_old) - fx((x, y), x_old)) / dx
    dfdx2 = (fx((x, y + dy), x_old) - fx((x, y), x_old)) / dy

    df = np.hstack((dfdx1, dfdx2))

    return df


def cost(x, y, measurement, var):
    p, r, x_old, u = measurement

    f = fx((x, y), x_old)

    b = np.array([[p], [r], [u[0, 0]], [u[1, 0]]])

    J = f - b

    W = var.T @ var + 0.1 * np.eye(4)
    c = J.T @ np.linalg.inv(W) @ J
    return c


def gradDescent(state, measurement, var):
    p, r, x_old, u = measurement
    x, y = state

    X = np.array([[x], [y]])

    b = np.array([[p], [r], [u[0, 0]], [u[1, 0]]])

    states = [(x, y)]
    for i in range(100):
        dfdx = partialf((x, y), x_old)

        f = fx((x, y), x_old)

        W = var.T @ var + 0.1 * np.eye(4)
        invW = np.linalg.inv(W)
        deltaX = np.linalg.inv(dfdx.T @ invW @ dfdx) @ dfdx.T @ (b - f)

        X = X + 5 * deltaX

        x = X[0, 0]
        y = X[1, 0]

        states.append((x, y))

    return states


def costContour(x, y, m, var):
    J = np.zeros((len(x), len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            J[j, i] = cost(x[i], y[j], m, var)
    return J


def prediction(state, u):
    u = np.reshape(u, (2, 1))
    guess = state + u
    return guess

In [20]:
# create environment
plt.figure(2, figsize=(10, 5))
x = np.linspace(0, 100, 40)
y = np.linspace(0, 50, 40)
[X, Y] = np.meshgrid(x, y)
g = ground(x)

# initial state
state = np.array([[5.0], [10.0]])
var = np.array([[1.0, 0.1, 0.5, 0.5]])


# control commands
t = 40
u1 = 2 * np.ones(t)
u2 = np.sin(4 * np.arange(t) / t)
us = np.vstack((u1, u2))


# store previous states
prev = [(state[0, 0], state[1, 0])]
prev_pred = [(state[0, 0], state[1, 0])]

wind = +1.0

# find cost contours every step
for i in range(len(u1) - 1):
    # predictions and control commands
    guess = prediction(state, us[:, i])
    u = np.reshape(us[:, i], (2, 1))
    state += u + np.random.normal(0, scale=var[0, 3], size=(2, 1))
    state[0, 0] -= np.random.normal(wind, scale=wind / 2)

    # measurements
    p = height2pressure(state[1, 0])
    r = state[1, 0] - ground(state[0, 0])
    m = (
        p + np.random.normal(0, scale=var[0, 0]),
        r + np.random.normal(0, scale=var[0, 1]),
        prev[i],
        u,
    )

    # store ground truth
    prev.append((state[0, 0], state[1, 0]))

    # store prediction
    sol = gradDescent((guess[0, 0] - 10, guess[1, 0] + 10), m, var)
    sx, sy = zip(*sol)
    prev_pred.append((sx[-1], sy[-1]))

    # plot new data
    plt.cla()

    # plot measurements
    h = pressure2height(p=m[0], var=0)
    plt.plot([0, np.max(x)], [h, h], "--", color=[0, 1, 1])
    plt.plot([state[0], state[0]], [state[1], state[1] - m[1]], "--", color=[0, 1, 0.5])
    plt.plot(state[0], state[1], "k*")

    # gradient descent
    plt.plot(sx[-1], sy[-1], "y*")
    # plt.plot(sx,sy,'r--')

    # ground truth
    prevx, prevy = zip(*prev)
    prevx_pred, prevy_pred = zip(*prev_pred)

    plt.plot(prevx[0] + sum(us[0, 0:i]), prevy[0] + sum(us[1, 0:i]), "ro")
    plt.plot(prevx, prevy, "k--")
    plt.plot(prevx_pred, prevy_pred, "y--")
    plt.legend(
        [
            "pressure measurement",
            "lidar measurement",
            "ground truth",
            "maximum likelihood estimate",
            "prediction without measurements",
        ]
    )

    # calculate cost function contour
    J = costContour(x, y, m, var)
    # minr,minc = np.where(J == np.amin(J))
    # plt.plot(x[minc],y[minr],'g*')
    plt.contourf(X, Y, J, 100, cmap="RdBu_r")
    plt.fill_between(x, 0, g, color="green")

    plt.xlabel("x-axis (m)")
    plt.ylabel("y-axis (m)")
    plt.title("Nonlinear Least Squares Drone Localization")
    # plt.axis('equal')
    plt.xlim([0, 100])
    plt.ylim([0, 40])

    plt.show()
    plt.pause(0.01)

In [21]:
diffxLS = np.array(prevx) - np.array(prevx_pred)
diffx = np.array(prevx) - prevx[0] - np.cumsum(us[0, :])

diffyLS = np.array(prevy) - np.array(prevy_pred)
diffy = np.array(prevy) - prevy[0] - np.cumsum(us[1, :])

all = np.vstack((diffxLS, diffx, diffyLS, diffy))
lim = np.max(abs(all))
plt.figure(3, figsize=(8, 4))
plt.plot(diffx, "b-")
plt.plot(diffxLS, "b--")
plt.plot(diffy, "r-")
plt.plot(diffyLS, "r--")
plt.legend(
    [
        "x-error - w/o measurements",
        "x-error - w/ measurements",
        "y-error - w/o measurements",
        "y-error - w/ measurements",
    ]
)
plt.ylim([-lim - 1, lim + 1])
plt.xlabel("time (s)")
plt.ylabel("position error (m)")
plt.grid("on")
plt.show()

print(np.std(diffx))
print(np.std(diffy))

print(np.std(diffxLS))
print(np.std(diffyLS))

10.883884615821461
1.9482745092118499
0.638757425792885
0.3966573070814091
